In [6]:
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

def preprocess_image(img):
    # Convert image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
    
    # Perform edge detection using Canny
    edges = cv2.Canny(blurred_img, 100, 200)
    
    # Add channel dimension to edges image
    edges_with_channel = np.expand_dims(edges, axis=-1)
    
    # Convert image to HSV color space
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    # Equalize histogram of the grayscale image
    equalized_img = cv2.equalizeHist(gray_img)
    
    # Add channel dimension to equalized image
    equalized_img_with_channel = np.expand_dims(equalized_img, axis=-1)
    
    return edges_with_channel, hsv_img, equalized_img_with_channel


def generate_augmented_images(input_dir, output_dir, datagen):
    classes = [class_name for class_name in os.listdir(input_dir) if os.path.isdir(os.path.join(input_dir, class_name))]
    
    for class_name in classes:
        class_input_dir = os.path.join(input_dir, class_name)
        class_output_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_output_dir, exist_ok=True)
        
        for filename in os.listdir(class_input_dir):
            img_path = os.path.join(class_input_dir, filename)
            
            # Skip non-image files
            if not any(img_path.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png', '.bmp']):
                print(f"Skipping non-image file: {img_path}")
                continue
            
            # Attempt to read the image
            img = cv2.imread(img_path)
            
            # Check if image reading was successful
            if img is None:
                print(f"Error: Unable to read image file: {img_path}")
                continue
            
            # Preprocess image
            edges_img, hsv_img, equalized_img = preprocess_image(img)
            
            # Add batch dimension
            edges_img = edges_img.reshape((1,) + edges_img.shape)
            hsv_img = hsv_img.reshape((1,) + hsv_img.shape)
            equalized_img = equalized_img.reshape((1,) + equalized_img.shape)
            
            # Generate augmented images for edge-detected version
            i = 0
            for batch in datagen.flow(edges_img, batch_size=1, save_to_dir=class_output_dir, save_prefix='aug_edges', save_format='jpg'):
                i += 1
                if i >= 20:  # Generate 20 augmented images per original image
                    break
            
            # Generate augmented images for HSV color space version
            i = 0
            for batch in datagen.flow(hsv_img, batch_size=1, save_to_dir=class_output_dir, save_prefix='aug_hsv', save_format='jpg'):
                i += 1
                if i >= 20:  # Generate 20 augmented images per original image
                    break
            
            # Generate augmented images for histogram equalized version
            i = 0
            for batch in datagen.flow(equalized_img, batch_size=1, save_to_dir=class_output_dir, save_prefix='aug_equalized', save_format='jpg'):
                i += 1
                if i >= 20:  # Generate 20 augmented images per original image
                    break

# Set directories
input_dir = r"C:\Users\VRAJ PATEL\Desktop\ipcv\Dataset\Dataset"  # Directory containing original images organized by class
output_dir = r"C:\Users\VRAJ PATEL\Desktop\ipcv\Final Dataset 2"  # Directory to save augmented images

# Define image data generator
datagen = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=(0.1, 0.9),
    shear_range=10.0,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    featurewise_center=True
)

# Generate augmented images for each class and save them
generate_augmented_images(input_dir, output_dir, datagen)


Skipping non-image file: C:\Users\VRAJ PATEL\Desktop\ipcv\Dataset\Dataset\Honda\desktop.ini
Skipping non-image file: C:\Users\VRAJ PATEL\Desktop\ipcv\Dataset\Dataset\Hyundai\desktop.ini
Skipping non-image file: C:\Users\VRAJ PATEL\Desktop\ipcv\Dataset\Dataset\Kia\desktop.ini
Skipping non-image file: C:\Users\VRAJ PATEL\Desktop\ipcv\Dataset\Dataset\Renault\desktop.ini
